In [1]:
import pandas as pd
import numpy as np


In [53]:
books=pd.read_csv("Books.csv")
ratings=pd.read_csv("Ratings.csv")
users=pd.read_csv("Users.csv")


<ipython-input-53-5040565c6482>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books=pd.read_csv("Books.csv")


# First check
    - Shape
    - null
    - Duplicate

In [4]:
print(books.shape)
print(ratings.shape)
print(users.shape)

(260212, 8)
(1149780, 3)
(278858, 3)


In [5]:
print(books.isnull().sum())

ISBN                   0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            4
dtype: int64


In [6]:
print(ratings.isnull().sum())


User-ID        0
ISBN           0
Book-Rating    0
dtype: int64


In [7]:
print(users.isnull().sum())  # age is missing almost 50% so we can dicard it


User-ID          0
Location         0
Age         110762
dtype: int64


In [8]:
books.duplicated().sum()

0

In [9]:
ratings.duplicated().sum()

0

In [10]:
users.duplicated().sum()

0

## Popularity Based

In [177]:
ratings_with_name=ratings.merge(books,on="ISBN")

In [178]:
rating_df=ratings_with_name.groupby(by="Book-Title").count()["Book-Rating"].reset_index()
rating_df.rename(columns={"Book-Rating":"rating_in_num"},inplace=True)

In [94]:
mean_rating=ratings_with_name.groupby(by="Book-Title")["Book-Rating"].mean().reset_index()
mean_rating.rename(columns={"Book-Rating":"mean_rating"},inplace=True)


In [95]:
# un books ki mean rating nikal lo jin pr more than 100 logo ne vote kra hy


In [96]:
rating_df=rating_df.merge(mean_rating,on="Book-Title")

In [120]:
top_rating=rating_df[rating_df['rating_in_num']>=250]
top_rating=top_rating.sort_values(by="mean_rating",ascending=False).head(50)  # top 50 utha li jin pr 250+ rating hy

In [126]:
# now i wan t author name image and publish ...

In [143]:
popular_books=top_rating.merge(books) # duplcates hain unko htao
popular_books=popular_books.drop_duplicates("Book-Title")[["Book-Title","Book-Author","Year-Of-Publication","Image-URL-M","rating_in_num","mean_rating"]]

In [144]:
popular_books.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50 entries, 0 to 195
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Book-Title           50 non-null     object 
 1   Book-Author          50 non-null     object 
 2   Year-Of-Publication  50 non-null     object 
 3   Image-URL-M          50 non-null     object 
 4   rating_in_num        50 non-null     int64  
 5   mean_rating          50 non-null     float64
dtypes: float64(1), int64(1), object(4)
memory usage: 2.7+ KB


# Collaborative Filtering Based Recommender

### Our approch would be take those users who voted at least 200 books means they are regular readers and take those books who has been voted by atleast 50
  ---------user1 user2 user3
  book1     3     0     3
  book2     4     7
  book3
..
,
,


In [166]:
ratings_with_name.groupby(by="User-ID").count()["Book-Rating"].reset_index()


In [196]:
x=ratings_with_name.groupby(by="User-ID").count()["Book-Rating"]>200
users_200plus_votes=x[x].index  #boolean idexing keeping only true values

In [211]:
mask=ratings_with_name["User-ID"].isin(users_200_votes)
user_filtered=ratings_with_name[mask]

# Books ki base r filter krna hy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [226]:
y=user_filtered.groupby(by="Book-Title").count()["User-ID"]>=50 # ab 200+ users se dekho kinti books hain jinko 50+ votes hain
books_50plus_voted=y[y].index  # movies name  agye jin pr more than 50 logo ne vote kra hy

In [225]:
books_50plus_voted

Index(['10 Lb. Penalty', '16 Lighthouse Road', '1984', '1st to Die: A Novel',
       '2010: Odyssey Two', '204 Rosewood Lane', '2061: Odyssey Three',
       '24 Hours', '2nd Chance', '3rd Degree',
       ...
       'YOU BELONG TO ME', 'Year of Wonders', 'You Belong To Me',
       'You Shall Know Our Velocity', 'Young Wives',
       'Zen and the Art of Motorcycle Maintenance: An Inquiry into Values',
       'Zoya', '\O\" Is for Outlaw"',
       '\Surely You're Joking, Mr. Feynman!\": Adventures of a Curious Character"',
       'stardust'],
      dtype='object', name='Book-Title', length=2444)

In [227]:
mask=user_filtered["Book-Title"].isin(books_50plus_voted)
books_filtered=user_filtered[mask]

In [232]:
pt=books_filtered.pivot_table(index="Book-Title",columns="User-ID",values="Book-Rating")
pt

User-ID,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
1st to Die: A Novel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2nd Chance,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN
4 Blondes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A Bend in the Road,0.0,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,9.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
You Belong To Me,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN


In [234]:
pt

User-ID,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Bend in the Road,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Now Find distance

In [317]:
from sklearn.metrics.pairwise import cosine_similarity
similarity=cosine_similarity(pt)
similarity.shape  # 706 books ka 706 books ke sath relation hy
# similarity ko enumerate kro take distance and index same rhain after sorting
# similarity=list(enumerate(similarity))

(706, 706)

In [331]:
def  recommend(book_name):
  suggestion=[]
  ind=np.where(pt.index==book_name)[0][0]
  print(ind)
  similar_items=sorted(list(enumerate(similarity[ind])),reverse=True,key=lambda x: x[1])[1:6]
  for i in similar_items:
    ind=i[0]
    temp_df=books[books["Book-Title"]==pt.index[ind]]
    suggestion.append(temp_df.drop_duplicates("Book-Title"))

    # print(ratings_with_name[ratings_with_name["Book-Title"]==pt.index[ind]])

  return suggestion
recommend("1984")

0


[           ISBN   Book-Title    Book-Author Year-Of-Publication Publisher  \
 285  0451526341  Animal Farm  George Orwell                2004    Signet   
 
                                            Image-URL-S  \
 285  http://images.amazon.com/images/P/0451526341.0...   
 
                                            Image-URL-M  \
 285  http://images.amazon.com/images/P/0451526341.0...   
 
                                            Image-URL-L  
 285  http://images.amazon.com/images/P/0451526341.0...  ,
             ISBN           Book-Title      Book-Author Year-Of-Publication  \
 1985  0449212602  The Handmaid's Tale  Margaret Atwood                1989   
 
           Publisher                                        Image-URL-S  \
 1985  Fawcett Books  http://images.amazon.com/images/P/0449212602.0...   
 
                                             Image-URL-M  \
 1985  http://images.amazon.com/images/P/0449212602.0...   
 
                                             Image-

In [269]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x: x[1])[0:5]

[(0, 0.9999999999999999),
 (47, 0.2702651417103732),
 (545, 0.2639619371123496),
 (82, 0.2366937434740099),
 (634, 0.23299389358170397)]

In [282]:
import pickle

In [313]:

pickle.dump(pt,open("pt.pkl","wb"))

In [333]:

pickle.dump(books,open("books.pkl","wb"))

In [300]:
popular_books.reset_index(inplace=True)